In [1]:
import numpy as np
import Orange
import os

In [2]:
import pandas as pd
df = pd.read_csv("test.csv")
display(df)

X = df.drop('y', axis=1).values
print(type(X))
display(X)

y = df['y'].values
print(type(y))
display(y)

,x1,x2,x3,x4,y
0,1,0,0,1,1
1,0,0,1,0,1
2,0,0,1,1,0
3,1,1,1,1,0


<class 'numpy.ndarray'>


array([[1, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [1, 1, 1, 1]])

<class 'numpy.ndarray'>


array([1, 1, 0, 0])

In [3]:
from IMLI import IMLI

imli = IMLI(n_clauses=1, lamda=1, solver="maxhs", n_partitions=2)
features = imli.generate_features(X, y, discretizer="entropy")
imli.fit(X, y, features)

In [4]:
preds = imli.predict(X)
print(imli.get_rule())

from sklearn.metrics import classification_report
print(classification_report(preds, y))

[(NOT X1)]
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.67      0.80         3

    accuracy                           0.75         4
   macro avg       0.75      0.83      0.73         4
weighted avg       0.88      0.75      0.77         4



In [5]:
imli = IMLI(n_clauses = 2, lamda = 2)
features = imli.generate_features(X, y)
imli.fit(X, y, features)
preds = imli.predict(X)
print(imli.get_rule())

from sklearn.metrics import classification_report
print(classification_report(preds, y))

# raise Exception("Berhenti saja hehehe")

[X0 OR (NOT X3)] AND [(NOT X1)]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



In [6]:
import pandas as pd
df = pd.read_csv("ionosphere.csv", header=None)
df = df.sample(frac=1)
# display(df.head())

import numpy as np
import Orange

df_numpy = df.values
X = df_numpy[:, :-1]
y = np.array([1 if u == 'g' else 0 for u in df_numpy[:, -1]])
domain = Orange.data.Domain.from_numpy(X, y)
ionosphere_table = Orange.data.Table.from_numpy(domain, X, y)

from IMLI import IMLI

imli = IMLI(n_clauses=2, lamda=2, solver="open-wbo", n_partitions=1)
proportion = 0.8
train_size = int(X.shape[0] * proportion)
print(train_size)
print(train_size)

features = imli.generate_features(X[:train_size], y[:train_size])
imli.fit(X[:train_size], y[:train_size], features)

preds = imli.predict(X[train_size:])
from sklearn.metrics import classification_report
print(classification_report(preds, y[train_size:]))

print(imli.get_rule())

280
280
              precision    recall  f1-score   support

           0       0.90      0.84      0.87        31
           1       0.88      0.93      0.90        40

    accuracy                           0.89        71
   macro avg       0.89      0.88      0.88        71
weighted avg       0.89      0.89      0.89        71

[X4 >= 0.418075 OR X22 >= 0.210400] AND [X26 < 0.999945]


In [7]:
for feature in imli.raw_features:
    print(feature.name)

X0
X1
X2
X3
X4
X5
X6
X7
X8
X9
X10
X11
X12
X13
X14
X15
X16
X17
X18
X19
X20
X21
X22
X23
X24
X25
X26
X27
X28
X29
X30
X31
X32
X33


In [8]:
from IMLI import DiscretizedFeature

for i, feature in enumerate(imli.new_features):
    print(feature.name)

X0
(NOT X0)
X1
(NOT X1)
X2 >= 0.190280
X2 < 0.190280
X3 >= -0.433090
X3 >= -0.000170
X3 >= 0.007075
X3 >= 0.563610
X3 < -0.433090
X3 < -0.000170
X3 < 0.007075
X3 < 0.563610
X4 >= 0.041440
X4 >= 0.418075
X4 >= 0.995175
X4 < 0.041440
X4 < 0.418075
X4 < 0.995175
X5 >= -0.769620
X5 >= -0.000715
X5 >= 0.001155
X5 >= 0.953815
X5 < -0.769620
X5 < -0.000715
X5 < 0.001155
X5 < 0.953815
X6 >= 0.029375
X6 >= 0.999760
X6 < 0.029375
X6 < 0.999760
X7 >= -0.395355
X7 >= -0.000375
X7 >= 0.000985
X7 >= 0.994515
X7 < -0.395355
X7 < -0.000375
X7 < 0.000985
X7 < 0.994515
X8 >= -0.704705
X8 >= -0.036395
X8 >= 0.011975
X8 >= 0.397340
X8 >= 0.994995
X8 < -0.704705
X8 < -0.036395
X8 < 0.011975
X8 < 0.397340
X8 < 0.994995
X9 >= -0.572990
X9 >= 0.989100
X9 < -0.572990
X9 < 0.989100
X10 >= -0.915760
X10 >= -0.010580
X10 >= 0.045765
X10 >= 0.999100
X10 < -0.915760
X10 < -0.010580
X10 < 0.045765
X10 < 0.999100
X11 >= -0.562930
X11 >= -0.000270
X11 >= 0.006365
X11 >= 0.981525
X11 < -0.562930
X11 < -0.000270
X11 < 0

In [9]:
print(imli.predict(X))
print(imli.get_rule())

[1 0 1 0 1 1 1 1 0 0 1 1 0 0 0 1 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 1 0 1 1 1
 1 1 1 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 1 1 1 1 1 0 1 1
 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 0 0 1 1 1 1 1 1 1 0 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1
 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1
 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 0 0 0 1 1
 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 0 1 0 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 1 1
 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 1 1 1]
[X4 >= 0.418075 OR X22 >= 0.210400] AND [X26 < 0.999945]


In [10]:
from IMLI import IMLI

import numpy as np

imli = IMLI()

X = np.array([
    [1.5, 1.5, 1.5, 1],
    [1.5, 1.5, 2, 0],
    [2.5, 1.5, 2, 1],
    [1.5, 1.5, 6, 1]
])

y = np.array([0, 1, 1, 0])

features = imli.generate_features(X, y)
imli.fit(X, y, features)
imli.get_rule()

'[(NOT X3)] AND [(NOT X3)] AND [(NOT X3)]'

In [11]:
for feature in imli.new_features:
    print(feature.name)
    
for feature in imli.raw_features:
    print(feature.name)

X3
(NOT X3)
X0
X1
X2
X3
